In [1]:
!pip install langchain langchain-community wtpsplit torch sentence-transformers sentencepiece protobuf==3.20.0 google-genai docling

In [2]:
from langchain.document_loaders import PyPDFLoader
from wtpsplit import SaT
import os
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
sat = SaT("sat-12l-sm")
sat.half().to(device)
#Sample PDF PATH
example_pdf_path = os.path.expanduser('../../cc_data/certs/reports/pdf/61da6b1582e593c8.pdf')

In [3]:
from docling.document_converter import DocumentConverter 
# for doing with gpu, set up some proper PipelineOptions and AcceleratorDevice: https://ds4sd.github.io/docling/examples/minimal_vlm_pipeline/
def to_text_docling(pdf_path: str) -> str:
    converter = DocumentConverter()
    result = converter.convert(pdf_path)
    return result.document.export_to_markdown()

/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling_core/types/doc/document.py:3339: DeprecationWarning: deprecated
  if not d.validate_tree(d.body) or not d.validate_tree(d.furniture):


In [4]:
to_text_docling(example_pdf_path)

/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling/pipeline/standard_pdf_pipeline.py:76: DeprecationWarning: Field `generate_table_images` is deprecated. To obtain table images, set `PdfPipelineOptions.generate_page_images = True` before conversion and then use the `TableItem.get_image` function.
  or self.pipeline_options.generate_table_images
/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling_core/types/doc/document.py:3339: DeprecationWarning: deprecated
  if not d.validate_tree(d.body) or not d.validate_tree(d.furniture):
/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling/pipeline/standard_pdf_pipeline.py:268: DeprecationWarning: Field `generate_table_images` is deprecated. To obtain table images, set `PdfPipelineOptions.generate_page_images = True` before conversion and then use the `TableItem.get_image` function.
  or self.pipeline_options.generate_table_images


"## TÜV Rheinland Nederland B.V.\n\n## Certification Report\n\n## FortiOS 6.4.9\n\nSponsor and developer:\n\nFortinet, Inc. 899 Kifer Road Sunnyvale, CA 94086 USA\n\nEvaluation facility:\n\nSGS Brightsight B.V. Brassersplein 2 2612 CT Delft The Netherlands\n\nReport number:\n\nNSCIB-CC-0587886-CR\n\nReport version:\n\n1\n\nProject number:\n\n0587886\n\nAuthor(s):\n\nAndy Brown\n\nDate:\n\n14 August 2023\n\nNumber of pages:\n\n12\n\nNumber of appendices:\n\n0\n\nReproduction of this report is authorised only if the report is reproduced in its entirety.\n\nTel. +31 (0)88 888 7 888 Fax +31 (0)88 888 7 879\n\n<!-- image -->\n\n<!-- image -->\n\n## CONTENTS\n\n| Foreword                       | Foreword                                    | 3   |\n|--------------------------------|---------------------------------------------|-----|\n| Recognition of the Certificate | Recognition of the Certificate              | 4   |\n| International recognition      | International recognition            

In [ ]:
def to_text_yakup(pdf_path: str) -> str:
    def preprocess_sent_segment(fulltext):
        no_new_line = fulltext.replace("\n", "")
        new_splitted = sat.split(no_new_line)
        return new_splitted

    loader = PyPDFLoader(os.path.join(pdf_path))
    documents = loader.load()
    texts = preprocess_sent_segment("".join([page.page_content for page in documents]))
    return '\n'.join(texts)

In [40]:
from google import genai
from google.genai import types
import pathlib
import httpx
try:
    api_key = open("../../gemini_api_key").read().strip()
    if not api_key:
        raise ValueError("API key file exists but is empty")
except FileNotFoundError:
    raise FileNotFoundError("Gemini API key file not found at ../../gemini_api_key")
client = genai.Client(api_key=api_key)
def to_text_gemini(pdf_path: str) -> str:
    filepath = pathlib.Path(pdf_path)
    if not filepath.exists():
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")
    prompt = "Fully transcribe this certificate document. Provide outputs in text. Do not prepend them with affirmations like 'Sure!', just start transcribing immediately. When you see a table, always transcribe it into a markdown table. Do not translate anything, just transcribe the text. For pagebreaks, use the '<page_break>' tag. If there is a drawing or a diagram, transcribe the text inside the diagram into image tags such that the content is inside, for example: <image>transcribed image content goes here</image>."
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[
                types.Part.from_bytes(
                    data=filepath.read_bytes(),
                    mime_type='application/pdf',
                ),
                prompt
            ]
        )
        return response.text
    except Exception as e:
        raise RuntimeError(f"Gemini API error: {str(e)}")

In [ ]:
from sec_certs.utils.pdf import convert_pdf_pdftotext
def to_text_seccerts(pdf_path: str) -> str:
    filepath = pathlib.Path(pdf_path)
    if not filepath.exists():
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")
    ocr, ok = convert_pdf_pdftotext(filepath, pathlib.Path("_temp_text.txt"))
    if ok:
        return open("_temp_text.txt").read()
    return ""

In [ ]:
gemini_result = to_text_gemini(example_pdf_path)

In [5]:
yakup_result = to_text_yakup(example_pdf_path)

In [52]:
seccerts_result = to_text_seccerts(example_pdf_path)

In [ ]:
print(gemini_result)

In [6]:
print(yakup_result)

TÜV Rheinland Nederland B.V.    Head Office: Westervoortsedijk 73 NL-6827 AV Arnhem P.O. Box 2220 NL-6802 CE Arnhem The Netherlands  Location Leek: Eiberkamp 10 NL-9351 VT Leek P.O. Box 37 NL-9350 AA Leek The Netherlands  info@nl.tuv.com www.tuv.com/nl Tel. +31 (0)88 888 7 888 Fax +31 (0)88 888 7 879  
TÜV Rheinland Nederland B.V. is a registered company at the Netherlands Chamber of Commerce (KVK), under number 27288788. 
VAT number: NL815820380B01 IBAN: NL61DEUT0265155096  Version 2022-02 ® TÜV, TUEV and TUV are registered trademarks. Any use or application requires prior approval.     
Certification Report   
FortiOS 6.4.9  Sponsor and developer: Fortinet, Inc. 899 Kifer Road  Sunnyvale, CA 94086  USA  Evaluation facility: SGS Brightsight B.V.  Brassersplein 2  2612 CT Delft  The Netherlands   
Report number: NSCIB-CC-0587886-CR   Report version: 1  Project number: 0587886  Author(s): Andy Brown  Date: 14 August 2023  Number of pages: 12  Number of appendices: 0        
Reproduction

In [53]:
print(seccerts_result)

TÜV Rheinland Nederland B.V.
Head Office:
Westervoortsedijk 73
NL-6827 AV Arnhem
P.O. Box 2220
NL-6802 CE Arnhem
The Netherlands
Location Leek:
Eiberkamp 10
NL-9351 VT Leek
P.O. Box 37
NL-9350 AA Leek
The Netherlands
info@nl.tuv.com
www.tuv.com/nl
Tel. +31 (0)88 888 7 888
Fax +31 (0)88 888 7 879
TÜV Rheinland Nederland B.V. is a
registered company at the
Netherlands Chamber of Commerce
(KVK), under number 27288788.
VAT number: NL815820380B01
IBAN: NL61DEUT0265155096
Version
2022-02
®
TÜV,
TUEV
and
TUV
are
registered
trademarks.
Any
use
or
application
requires
prior
approval.
Certification Report
FortiOS 6.4.9
Sponsor and developer: Fortinet, Inc.
899 Kifer Road
Sunnyvale, CA 94086
USA
Evaluation facility: SGS Brightsight B.V.
Brassersplein 2
2612 CT Delft
The Netherlands
Report number: NSCIB-CC-0587886-CR
Report version: 1
Project number: 0587886
Author(s): Andy Brown
Date: 14 August 2023
Number of pages: 12
Number of appendices: 0
Reproduction of this report is authorised only if the 

In [3]:
from sec_certs.dataset import CCDataset
from sec_certs.sample import CCCertificate
import pandas as pd

/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling_core/types/doc/document.py:3339: DeprecationWarning: deprecated
  if not d.validate_tree(d.body) or not d.validate_tree(d.furniture):


In [4]:
import json

In [5]:
import shutil
import json
import os
from pathlib import Path
from typing import List

src_folder = Path("../../cc_data")

target_digests = [
    "236b01ad914b9af4",
    "afff54c61c7bd4a0",
    "67b38ce4914a3c21",
    "36f5ed4050c8ae40",
    "66d6641328623e62",
    "91b3ea5f789bca05",
    "e295951359077a30",
    "33b2d4e7af811f9d",
    "e79b0851426ecf19",
    "5aff27b2a405d882"
]

dst_suffixes = ["pdftotext", "sat", "gemini", "docling"]

pdf_dirs = [
    "certs/certificates/pdf",
    "certs/reports/pdf",
    "certs/targets/pdf",
]

txt_dirs = [
    "certs/certificates/txt",
    "certs/certificates/txt_processed",
    "certs/reports/txt",
    "certs/reports/txt_processed",
    "certs/targets/txt",
    "certs/targets/txt_processed",
]

def preproc_dataset(dset_dict, topn=100, keep_certs: List[str] = None):
    if keep_certs is not None:
        dset_dict["certs"] = [c for c in dset_dict["certs"] if c["dgst"] in keep_certs]
    dset_dict["certs"] = dset_dict["certs"][:topn]
    dset_dict["n_certs"] = len(dset_dict["certs"])
    dset_dict["state"]["pdfs_converted"] = False
    for i in range(len(dset_dict["certs"])):
        for t in ["st", "report", "cert"]:
            dset_dict["certs"][i]["state"][t]["convert_ok"] = False

    return dset_dict

json_path = Path(src_folder) / "dataset.json"
with open(json_path) as json_file:
    json_data = json.load(json_file)
json_data = preproc_dataset(json_data, keep_certs=target_digests)

for suffix in dst_suffixes:
    dst_folder = Path(f"../../cc_data_{suffix}")
    if os.path.exists(dst_folder):
        print(f"{dst_folder} already exists, skipping...")
        continue
    dst_folder.mkdir(parents=True, exist_ok=True)

    dst_json_path = dst_folder / "dataset.json"
    with open(dst_json_path, "w") as json_file:
        json.dump(json_data, json_file, indent=4)

    # collect valid digests
    valid_digests = {cert["dgst"] for cert in json_data.get("certs", [])}
    
    for pdf_dir in pdf_dirs:
        src_pdf_path = src_folder / pdf_dir
        dst_pdf_path = dst_folder / pdf_dir
        dst_pdf_path.mkdir(parents=True, exist_ok=True)
        
        if src_pdf_path.exists():
            for pdf_file in src_pdf_path.glob("*.pdf"):
                if pdf_file.stem in valid_digests:
                    shutil.copy2(pdf_file, dst_pdf_path)


../../cc_data_pdftotext already exists, skipping...
../../cc_data_sat already exists, skipping...
../../cc_data_gemini already exists, skipping...
../../cc_data_docling already exists, skipping...


In [6]:
from pypdf import PdfReader
from pathlib import Path

def count_pdf_pages(pdf_path):
    reader = PdfReader(str(pdf_path))
    return len(reader.pages)

def count_total_pages(pdf_dirs, base_path):
    total_pages = 0
    for pdf_dir in pdf_dirs:
        pdf_path = Path(base_path) / pdf_dir
        if pdf_path.exists():
            for pdf_file in pdf_path.glob("*.pdf"):
                total_pages += count_pdf_pages(pdf_file)
    return total_pages

In [7]:
# example usage
for suffix in ["pdftotext", "sat", "gemini", "docling"]:
    path = f"../../cc_data_{suffix}"
    pages = count_total_pages(pdf_dirs, path)
    print(f"Total pages in {path}: {pages}")

Total pages in ../../cc_data_pdftotext: 847
Total pages in ../../cc_data_sat: 847
Total pages in ../../cc_data_gemini: 847
Total pages in ../../cc_data_docling: 847


In [8]:
set(dst_suffixes) - {"sat"}

{'docling', 'gemini', 'pdftotext'}

In [9]:
from sec_certs.utils.pdf import PDFConversionMethod
suffix_to_method = {
    "pdftotext": PDFConversionMethod.PDFTOTEXT,
    "sat": PDFConversionMethod.SEGMENTATION_TRANSFORMER,
    "gemini": PDFConversionMethod.VLM,
    "docling": PDFConversionMethod.DOCLING
}
dset = None
# no request throttling implemented, so had to run it with gemini several times to get past the free tier limits.
for suffix in set(dst_suffixes) - {"sat"}: # sat is too taxing to run with no GPU
    curr_folder = Path(f"../../cc_data_{suffix}")
    print(f"processing {suffix}")
    dset = CCDataset.from_json(curr_folder / "dataset.json")
    dset.convert_all_pdfs(fresh=False, conversion_method=suffix_to_method[suffix])
# print(dset)

Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.


processing docling


Converting PDFs of security targets to txt:   0%|          | 0/1 [00:00<?, ?it/s]

/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling/pipeline/standard_pdf_pipeline.py:76: DeprecationWarning: Field `generate_table_images` is deprecated. To obtain table images, set `PdfPipelineOptions.generate_page_images = True` before conversion and then use the `TableItem.get_image` function.
  or self.pipeline_options.generate_table_images


KeyboardInterrupt: 

/home/fstomakhin/Documents/CHESS/sec-certs/venv/lib/python3.10/site-packages/docling_core/types/doc/document.py:3339: DeprecationWarning: deprecated
  if not d.validate_tree(d.body) or not d.validate_tree(d.furniture):
Error when converting pdf->txt: Merged element must be of same type as element.
Cert dgst: 67b38ce4914a3c21 failed to convert st pdf->txt


In [10]:
import sec_certs.configuration as config_module
config_module.config.load_from_yaml("processing_config.yaml")

In [ ]:
dsets = {}

In [ ]:
for suffix in set(dst_suffixes) - {"sat"}:
    curr_folder = Path(f"../../cc_data_{suffix}")
    shutil.copytree("../../cc_data/auxiliary_datasets", curr_folder / "auxiliary_datasets", dirs_exist_ok=True)
    print(f"processing {suffix}")
    dset = CCDataset.from_json(curr_folder / "dataset.json")
    dset.analyze_certificates()
    dsets[suffix] = dset
    shutil.rmtree(curr_folder / "auxiliary_datasets")

Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.
Method auxiliary_datasets_dir can only be called on backed dataset.


processing docling


Extracting report metadata:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting target metadata:   0%|          | 0/9 [00:00<?, ?it/s]

Extracting cert metadata:   0%|          | 0/4 [00:00<?, ?it/s]

...

Fitting the CPE classifier:   0%|          | 0/1197837 [00:00<?, ?it/s]

Predicting CPE matches with the classifier: 100%|██████████| 10/10 [00:00<00:00, 39.64it/s]


Expanding and filtering criteria configurations:   0%|          | 0/37948 [00:00<?, ?it/s]

Building-up lookup dictionaries for fast CVE matching:   0%|          | 0/243101 [00:00<?, ?it/s]

Computing related CVES: 0it [00:00, ?it/s]


In [15]:
from sec_certs.utils.pdf import PDFConversionMethod

In [ ]:
dset.convert_all_pdfs()

Converting PDFs of certification reports to txt:   0%|          | 0/6172 [00:00<?, ?it/s]

Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/52cef68b9ccac840.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/65fb86b0b7259ffb.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/a80051397468fac6.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/8421233fb00f1cd5.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/f7c1b24c776136a8.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/70c176a3a9ae4afe.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/repo

KeyboardInterrupt: 

Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/f77d90052ead7dcc.pdf
Detected garbage during conversion of /home/fstomakhin/Documents/CHESS/sec-certs/notebooks/cc/../../cc_data/certs/reports/pdf/0752dc8f98749dce.pdf


In [ ]:
cert = next(iter(dset))

True

In [ ]:
dset = CCDataset()
dset.get_certs_from_web()
dset.process_auxiliary_datasets()
dset.download_all_artifacts()
dset.convert_all_pdfs()
dset.analyze_certificates()